In [1]:
import pandas as pd

In [2]:
orders = pd.read_csv("splitted_events/orders.csv")
orders.head()

,user_id,site_id,account_id,order_action,date,source
0,150-3302,150,3302,UP,2012-03-05,user
1,150-3302,150,3302,DW,2011-07-19,user
2,150-3302,150,3302,IN,2011-05-04,user
3,150-3302,150,3302,NP,2015-07-01,user
4,150-3302,150,3302,UP,2013-08-20,user


In [3]:
orders.site_id.value_counts()

400    77269
811    63421
150    46807
106    46528
Name: site_id, dtype: int64

In [4]:
orders.order_action.value_counts()

UP    58156
TC    57113
NP    36646
DW    32550
SR    22024
IN    20826
DI     4960
RS     1750
Name: order_action, dtype: int64

In [5]:
orders.source.value_counts()

user    234025
Name: source, dtype: int64

#### Выясним, нет ли разных пользователей с повторяющимися account_id

In [6]:
for acc in orders.account_id.unique():
    sites = orders[orders.account_id == acc].site_id
    if len(sites.unique()) > 1:
        print 'account: ', acc, ' is not unique'


account:  87338602  is not unique
account:  604186501  is not unique
account:  604735601  is not unique
account:  611700802  is not unique
account:  616358701  is not unique


Как видим, такие пользователи есть.

In [7]:
calls = pd.read_csv("splitted_events/calls.csv")
calls.head()

,user_id,site_id,account_id,call,date,source
0,150-3302,150,3302,Balance Question,2015-09-21,user
1,106-22403,106,22403,TV Channel Problem/Tiling,2015-02-14,user
2,106-32501,106,32501,Where is Tech,2015-02-13,user
3,106-76201,106,76201,Make Payment,2015-06-01,user
4,106-76201,106,76201,Fix Outside Wires,2015-07-15,user


In [8]:
calls.call.value_counts()

Bill Question                           11306
Balance Question                         5237
Make Payment                             2486
Service not working                      2007
Need Bill Extension                      1128
Past Due                                  868
Where is Tech                             669
Reschedule Appt                           668
Check on Status                           532
Need Bill                                 363
Change Info                               312
Ghost Call                                202
Full Account Disconnect                   104
Downgrade                                  90
How to use feature                         87
TV Channel Problem/Tiling                  77
New Service                                70
Reduce My Bill                             54
Fix Inside Wires                           53
Had Feedback & Questions                   52
Trouble Installing Self- Install kit       47
Fix Outside Wires                 

In [8]:
calls = calls.rename(columns={"call" : "action"})

In [9]:
calls.head()

,user_id,site_id,account_id,action,date,source
0,150-3302,150,3302,Balance Question,2015-09-21,user
1,106-22403,106,22403,TV Channel Problem/Tiling,2015-02-14,user
2,106-32501,106,32501,Where is Tech,2015-02-13,user
3,106-76201,106,76201,Make Payment,2015-06-01,user
4,106-76201,106,76201,Fix Outside Wires,2015-07-15,user


In [10]:
orders = orders.rename(columns={"order_action":"action"})
orders.head()

,user_id,site_id,account_id,action,date,source
0,150-3302,150,3302,UP,2012-03-05,user
1,150-3302,150,3302,DW,2011-07-19,user
2,150-3302,150,3302,IN,2011-05-04,user
3,150-3302,150,3302,NP,2015-07-01,user
4,150-3302,150,3302,UP,2013-08-20,user


In [11]:
order_calls = pd.concat([orders, calls])

In [12]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234025 entries, 0 to 234024
Data columns (total 6 columns):
user_id       234025 non-null object
site_id       234025 non-null int64
account_id    234025 non-null int64
action        234025 non-null object
date          234025 non-null object
source        234025 non-null object
dtypes: int64(2), object(4)
memory usage: 10.7+ MB


In [13]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26979 entries, 0 to 26978
Data columns (total 6 columns):
user_id       26979 non-null object
site_id       26979 non-null int64
account_id    26979 non-null int64
action        26979 non-null object
date          26979 non-null object
source        26979 non-null object
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [14]:
order_calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261004 entries, 0 to 26978
Data columns (total 6 columns):
user_id       261004 non-null object
site_id       261004 non-null int64
account_id    261004 non-null int64
action        261004 non-null object
date          261004 non-null object
source        261004 non-null object
dtypes: int64(2), object(4)
memory usage: 13.9+ MB


In [15]:
oc_sorted = order_calls.sort_values(['user_id', 'date'])
oc_sorted[:10]

,user_id,site_id,account_id,action,date,source
97457,106-100000503,106,100000503,IN,2011-02-15,user
97458,106-100000503,106,100000503,UP,2011-02-18,user
97455,106-100000503,106,100000503,UP,2014-02-11,user
97456,106-100000503,106,100000503,UP,2015-03-16,user
9327,106-100000503,106,100000503,Bill Question,2015-03-24,user
9328,106-100000503,106,100000503,Bill Question,2015-03-24,user
97475,106-100010603,106,100010603,UP,2008-12-10,user
97476,106-100010603,106,100010603,TC,2008-12-29,user
97477,106-100010603,106,100010603,TC,2009-01-02,user
97478,106-100010603,106,100010603,TC,2009-02-07,user


In [19]:
##oc_sorted.to_csv('splitted_events/orders_with_calls.csv', index=False)
case_id = oc_sorted.user_id.copy()
to_prom = oc_sorted
# to_prom.insert(0, 'case_id', case_id)
to_prom = to_prom.rename(columns={'source':'initiator', 'user_id':'source', 'action':'activity'})
to_prom.head()

,case_id,source,site_id,account_id,activity,date,initiator
97457,106-100000503,106-100000503,106,100000503,IN,2011-02-15,user
97458,106-100000503,106-100000503,106,100000503,UP,2011-02-18,user
97455,106-100000503,106-100000503,106,100000503,UP,2014-02-11,user
97456,106-100000503,106-100000503,106,100000503,UP,2015-03-16,user
9327,106-100000503,106-100000503,106,100000503,Bill Question,2015-03-24,user


In [20]:
to_prom.to_csv("splitted_events/to_prom_orders.csv", index=False)